<a href="https://colab.research.google.com/github/hyesukim1/Face-Verification-Project/blob/main/facenet_%EC%88%98%EC%A0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from keras.models import load_model

model = load_model('/content/drive/MyDrive/keras-facenet/model/facenet_keras.h5')

print(model.inputs)
print(model.outputs)

In [ ]:
pip install mtcnn

In [ ]:
pip install keras_facenet

In [ ]:
from fileinput import filename
import os
from os import listdir
import scipy
import cv2
import dlib 
import numpy as np
from PIL import Image # 이미지 크기 조정 라이브러리
import matplotlib.pyplot as plt
import tensorflow as tf 
from mtcnn.mtcnn import MTCNN #불러온 사진에 얼굴 감자히는 '얼굴 감지기'라이브러리
import keras
from keras.models import load_model
from keras_facenet import FaceNet
from tensorflow.keras.models import Model
from numpy import savez_compressed
from numpy import expand_dims
from sklearn import preprocessing# 3명의 유명인사 얼굴 데이터셋으로 분류기 개발
from numpy import load
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC

In [ ]:
def extract_face(filename, input_size=(160, 160)):
  
  # 이미지 읽고 컬러 맞추기
  image = cv2.cvtColor(cv2.imread(filename), cv2.COLOR_BGR2RGB)
  
  # mctnn으로 각 이미지 랜드마크 및 경계 박스 얻기(face detection)
  detector = MTCNN()
  results = detector.detect_faces(image)
  x1, y1, width, height = results[0]['box']
  x2, y2 = x1 + width, y1 + height

  # 전체 사진에서 얼굴만 배열로 추출
  pixels = np.array(image)
  face = pixels[y1:y2, x1:x2]

  # 얼굴만 자른 사진이 모델로 들어갈 수 있게 resize
  img = Image.fromarray(face) # 배열 객체를 입력으로 받아 배열 객체에서 만든 이미지 객체를 반환
  img = img.resize(input_size)
  face_array = np.asarray(img)
  return face_array

In [ ]:
def load_faces(directory):
  # 파일 안에 모든 이미지 불러오고 모든 input 이미지에서 얼굴 추출
  faces = []
  for filename in listdir(directory):
    path = directory + filename
    face = extract_face(path)
    faces.append(face)
  return faces

In [ ]:
# 하나의 얼굴에 얼굴 임베딩 얻기(facenet 모델을 사용해서 얼굴영역을 임베딩 벡터로 변환)
def get_face_embedding(model, face_pixels):
  face_pixels = face_pixels.astype('int32')
  mean, std = face_pixels.mean(), face_pixels.std()
  face_pixels = (face_pixels - mean) / std
  samples = expand_dims(face_pixels, axis=0)
  face_embedding = model.predict(samples)
  return face_embedding

In [ ]:
img_input = '/content/drive/MyDrive/facedataset/train/김희애/김희애.jpg'
img_test = '/content/drive/MyDrive/facedataset/test/김희애/김희애test.jpg'

first_image_file = extract_face(img_input)
second_image_file = extract_face(img_test)
first_image_file

In [ ]:
# input_image embedding 구하기
input_image = []
embedding = get_face_embedding(model, first_image_file)
input_image.append(embedding)
input_image = np.asarray(input_image)

# compare_image embedding 구하기
compare_image = []
embedding = get_face_embedding(model, second_image_file)
compare_image.append(embedding)
compare_image = np.asarray(compare_image)

In [ ]:
dist = np.linalg.norm(input_image - compare_image) # 유클리디안 거리 계산
print(dist)